# Tutorial

Start by downloading the astrometry data to the local working folder:

In [1]:
!wget https://raw.githubusercontent.com/wbalmer/backtracks/main/tests/scorpions1b_orbitizelike.csv

--2023-12-19 10:22:35--  https://raw.githubusercontent.com/wbalmer/backtracks/main/tests/scorpions1b_orbitizelike.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 831 [text/plain]
Saving to: ‘scorpions1b_orbitizelike.csv’

scorpions1b_orbitiz 100%[===================>]     831  --.-KB/s    in 0s      

2023-12-19 10:22:35 (99.1 MB/s) - ‘scorpions1b_orbitizelike.csv’ saved [831/831]



In [2]:
from backtracks import System

In [3]:
track = System(target_name="HD 131399A",
               candidate_file="scorpions1b_orbitizelike.csv",
               nearby_window=0.5,
               fileprefix='./')

[BACKTRACK INFO]: Resolved the target star 'HD 131399A' in Simbad!
[BACKTRACK INFO]: Resolved target's Gaia ID from Simbad, Gaia DR3 6204835284262018688
INFO: Query finished. [astroquery.utils.tap.core]
[BACKTRACK INFO]: gathered target Gaia data
INFO: Query finished. [astroquery.utils.tap.core]
[BACKTRACK INFO]: gathered 6445 Gaia objects from the 0.5 sq. deg. nearby HD 131399A
[BACKTRACK INFO]: Finished nearby background gaia statistics
[BACKTRACK INFO]: Queried distance prior parameters, L=2524.23, alpha=1.05, beta=0.62
[BACKTRACK INFO]: Opened ephemeris file
[BACKTRACK INFO]: made cat entry for host
[BACKTRACK INFO]: transformed cat entry for host


In [4]:
results = track.fit(dlogz=0.5, npool=4, dynamic=False, nlive=200, mpi_pool=False, resume=False)

[BACKTRACK INFO]: Beginning sampling, I hope


iter: 4781 | +200 | bound: 50 | nc: 1 | ncall: 21942 | eff(%): 22.910 | loglstar:   -inf < -100.256 <    inf | logz: -123.364 +/-  0.336 | dlogz:  0.002 >  0.500                                     

In [5]:
track.save_results(fileprefix='./')

In [6]:
track.load_results(fileprefix='./')

In [7]:
%%capture

fig_track, fig_post, fig_diag, fig_prior, fig_hood = \
    track.generate_plots(days_backward=3.*365.,
                         days_forward=3.*365,
                         step_size=100.,
                         ref_epoch=None,
                         plot_radec=False,
                         plot_stationary=True,
                         fileprefix='./',
                         filepost='.png')

In [8]:
! ls *png

HD_131399A_bjprior_backtracks.png       HD_131399A_model_backtracks.png
HD_131399A_corner_backtracks.png        HD_131399A_nearby_gaia_distribution.png
HD_131399A_evidence_backtracks.png
